# CPIT 440 lab assignment

In the housing dataset, we detected some problems such as capping the `median_house_value` at 500001. In addition, we did not remove the outliers.  
1. remove all the districts whose price equals 500001
2. remove outliers
3. repeat all the preprocessing steps
4. use the `RandomForestRegressor` again and use the cross validation
5. what is the importance of each feature based on the cross validation
6. evaluate the model on the test set, and comment on the new score

-------------------------------------------

In [1]:
import pandas as pd
#url="https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv"
url="/Users/nooralsharif/CPIT-440 LAB/LAB work/p2 housing.csv"
housing=pd.read_csv(url)


print(housing)

       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0        -122.23     37.88                41.0        880.0           129.0   
1        -122.22     37.86                21.0       7099.0          1106.0   
2        -122.24     37.85                52.0       1467.0           190.0   
3        -122.25     37.85                52.0       1274.0           235.0   
4        -122.25     37.85                52.0       1627.0           280.0   
...          ...       ...                 ...          ...             ...   
20635    -121.09     39.48                25.0       1665.0           374.0   
20636    -121.21     39.49                18.0        697.0           150.0   
20637    -121.22     39.43                17.0       2254.0           485.0   
20638    -121.32     39.43                18.0       1860.0           409.0   
20639    -121.24     39.37                16.0       2785.0           616.0   

       population  households  median_income  media

## 1. Remove all the districts whose price equals 500001

In [2]:
capped_index = (housing['median_house_value'] == 500001)
#********************************
# write the code that remove districts with price = 500001
#********************************
#housing.drop(capped_index.index, inplace = True)
#housing.reset_index(drop=True, inplace=True)

index_numbers = capped_index[capped_index].index
housing.drop(index_numbers, inplace = True)
housing.reset_index(drop=True, inplace=True)


print(housing)

       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0        -122.23     37.88                41.0        880.0           129.0   
1        -122.22     37.86                21.0       7099.0          1106.0   
2        -122.24     37.85                52.0       1467.0           190.0   
3        -122.25     37.85                52.0       1274.0           235.0   
4        -122.25     37.85                52.0       1627.0           280.0   
...          ...       ...                 ...          ...             ...   
19670    -121.09     39.48                25.0       1665.0           374.0   
19671    -121.21     39.49                18.0        697.0           150.0   
19672    -121.22     39.43                17.0       2254.0           485.0   
19673    -121.32     39.43                18.0       1860.0           409.0   
19674    -121.24     39.37                16.0       2785.0           616.0   

       population  households  median_income  media

## 2. Remove outliers

In [3]:
#****************This section is complete**********************
housing_num = housing.drop("ocean_proximity", axis=1)
Q1 = housing_num.quantile(0.25)
Q3 = housing_num.quantile(0.75)
IQR = Q3 - Q1

outlier_index = ((housing_num < (Q1 - 1.5 * IQR)) |(housing_num > (Q3 + 1.5 * IQR))).any(axis=1)
index_numbers = housing_num[outlier_index].index
housing.drop(index_numbers,inplace = True)
housing.reset_index(drop=True, inplace=True)

**convert `ocean_proximity` from string to numerical categories**

In [4]:
#****************This section is complete**********************
housing_cat = housing["ocean_proximity"]
housing_cat_encoded, housing_categories = housing_cat.factorize()
housing_cat =pd.DataFrame(housing_cat_encoded,columns=["ocean_proximity"])
housing["ocean_proximity"]=housing_cat


## 3. Preprocessing

### 3.1 Splitting into train and test sets

In [5]:
#****************This section is complete**********************
import numpy as np
housing["income_cat"] = np.ceil(housing["median_income"]*(2/3))
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)

from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42,stratify=housing['income_cat'])

train_set=train_set.drop("income_cat", axis=1)
test_set=test_set.drop("income_cat", axis=1)

train_set.reset_index(drop=True, inplace=True)
test_set.reset_index(drop=True, inplace=True)

### 3.2 Divide the dataset into Dependent & Independent variable

In [6]:
#********************************
# write the division code here
train_X = train_set.drop("median_house_value", axis=1)
train_y = train_set["median_house_value"].copy()
test_X = train_set.drop("median_house_value", axis=1)
test_y = train_set["median_house_value"].copy()
#********************************

### 3.3 Data cleaning (Missing values)

In [7]:
housing=train_X
from sklearn.impute import SimpleImputer
import numpy as np
imputer = SimpleImputer(missing_values=np.nan,strategy='median')
housing_num = housing.drop("ocean_proximity", axis=1)
#***********************************************************
# There are two missed lines of code here
imputer.fit(housing_num)
housing_num.median().values
X = imputer.transform(housing_num)
#***********************************************************
housing_tr = pd.DataFrame(X, columns=housing_num.columns)

### 3.4  Handeling the categorical attributes

In [8]:
#****************This section is complete**********************
housing_cat = housing["ocean_proximity"]
ocean_ndarray = housing_cat.to_numpy()

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
housing_cat_1hot = encoder.fit_transform(ocean_ndarray.reshape(-1,1))
housing_cat_1hot=housing_cat_1hot.toarray()
housing_cat =pd.DataFrame(housing_cat_1hot,columns=housing_categories.tolist())

### 3.5 Data scaling 

In [9]:
#****************This section is complete**********************
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(housing_tr)
scaled=scaler.transform(housing_tr)
housing_scaled = pd.DataFrame(scaled, columns=housing_tr.columns)
train_X = pd.concat([housing_scaled, housing_cat], axis=1)

-----------------------------------------------

In the follwoing cell I will repeat the steps to be implemented on the test set. Note that I will take the imputer, encodr, and scaler as fitted from the train data and use them to transform the test set to prevent data leakage.

In [10]:
#****************This section is complete**********************
#missing values
housing = test_X
housing_num = housing.drop("ocean_proximity", axis=1)
X = imputer.transform(housing_num)
housing_tr = pd.DataFrame(X, columns=housing_num.columns)
#categorical attribute
housing_cat = housing["ocean_proximity"]
ocean_ndarray = housing_cat.to_numpy()
housing_cat_1hot = encoder.transform(ocean_ndarray.reshape(-1,1))
housing_cat_1hot=housing_cat_1hot.toarray()  #from sparse to dense
housing_cat =pd.DataFrame(housing_cat_1hot,columns=housing_categories.tolist()) 
#standardization
scaled=scaler.transform(housing_tr)
housing_scaled = pd.DataFrame(scaled, columns=housing_tr.columns)
#concatenation of numerical and categorical
test_X = pd.concat([housing_scaled, housing_cat], axis=1)

## 4.  `RandomForestRegressor`  and  cross-validation

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error

forest_reg = RandomForestRegressor(n_estimators=100 )
#***********************************************************
# There is one missed line of code here 
forest_reg.fit(train_X, train_y.values.ravel())

#Now we will evaluate the model.
#housing_predictions = forest_reg.predict(train_X)
#forest_mse = mean_squared_error(train_y, housing_predictions)
#forest_rmse = np.sqrt(forest_mse)
#forest_rmse
#***********************************************************

forest_output = cross_validate(forest_reg, train_X, train_y.values.ravel(), scoring="neg_mean_squared_error", cv=10,  return_estimator =True)
forest_rmse_scores = np.sqrt(-forest_output["test_score"])

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(forest_rmse_scores)


Scores: [41777.77838243 41517.89645069 40018.81611014 40844.40339007
 42636.79559235 41985.56110018 40571.32323066 41996.83713962
 41221.93303378 39096.50757272]
Mean: 41166.78520026484
Standard deviation: 1004.1081260139713


## 5. feature_importances based on cross_validation

In [12]:
#****************This section is complete**********************
feature_importances=np.zeros(len(train_X.columns)) 
for estimator in forest_output['estimator']:
    feature_importances=np.add(feature_importances,estimator.feature_importances_)

feature_importances=np.divide(feature_importances,10)   # because cv=10, hence we calculate the mean
sorted(zip(feature_importances, train_X.columns), reverse=True)

[(0.33727922686256734, 'median_income'),
 (0.24370220581556076, 'INLAND'),
 (0.12611563899767936, 'longitude'),
 (0.11272885341490062, 'latitude'),
 (0.048022809214560626, 'housing_median_age'),
 (0.03988685640641344, 'population'),
 (0.032787775922998155, 'total_rooms'),
 (0.026080816035829187, 'total_bedrooms'),
 (0.023589754170590938, 'households'),
 (0.006370166483653951, 'NEAR OCEAN'),
 (0.0024924626612509004, '<1H OCEAN'),
 (0.000888013852585532, 'NEAR BAY'),
 (5.542016140929111e-05, 'ISLAND')]

## 6. comment on the new score you got on the test set

In [13]:
#***********************************************************
# There are two missed lines of code here 
final_predictions = forest_reg.predict(test_X)
final_mse = mean_squared_error(test_y, final_predictions)
#***********************************************************
final_rmse = np.sqrt(final_mse)
final_rmse

15304.096696363715